In [18]:
import openai
import os
import csv
import pandas as pd
from evaluate import load
openai.api_key = os.getenv('OPENAI_API_KEY') 


def create_question(row):
    try:
        text = row['Text']
        concept = row['concepts']
        response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
        {
            "role": "user",
            "content": f"Use this course text: {text} as well as this concept: {concept} to generate a single question with no additional text and its answer separated by a tab character."
        }
        ],
        temperature=0.2,
        max_tokens=256,
        top_p=0.1,
        frequency_penalty=0,
        presence_penalty=0
        )
        question = response["choices"][0]["message"]["content"]
        

        return question
    
    except Exception as e:
        print(f"Error: {e}")
        return 'Error'

In [2]:
df = pd.read_csv('f_22_oli_content_with_concepts.csv')

In [3]:
display(df.head())

,Unit,Module,Title,Text,Subheaders,concepts
0,Deep Learning and Model Deployment,Model Deployment,Quiz 10,NaN,NaN,"['Data science', 'Knowledge assessment', 'Key ..."
1,Data Science Project Planning,Developing a Vision,Module 4 Summary,This module focuses on the foundation of a doc...,NaN,"['Foundation of a documentation set', 'Vision ..."
2,Analytic Algorithms and Model Building,Model Selection,Model Selection for Prediction,To replicate the setting of performing predict...,NaN,['Replicating the setting of performing predic...
3,Data Science Project Planning,Requirements Gathering,Successful Requirements Gathering,The requirements gathering process is not line...,Validating Requirements,['Requirements gathering process is not linear...
4,Exploratory Data Analysis,Feature Engineering,t-SNE,"In the last section, we explored Principal Com...",References:,Error


In [4]:
df['concepts'] = df['concepts'].apply(lambda x: [] if x == 'Error' else x)
import ast
df['concepts'] = df['concepts'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)


In [5]:
display(df.head())

,Unit,Module,Title,Text,Subheaders,concepts
0,Deep Learning and Model Deployment,Model Deployment,Quiz 10,NaN,NaN,"[Data science, Knowledge assessment, Key conce..."
1,Data Science Project Planning,Developing a Vision,Module 4 Summary,This module focuses on the foundation of a doc...,NaN,"[Foundation of a documentation set, Vision doc..."
2,Analytic Algorithms and Model Building,Model Selection,Model Selection for Prediction,To replicate the setting of performing predict...,NaN,[Replicating the setting of performing predict...
3,Data Science Project Planning,Requirements Gathering,Successful Requirements Gathering,The requirements gathering process is not line...,Validating Requirements,[Requirements gathering process is not linear....
4,Exploratory Data Analysis,Feature Engineering,t-SNE,"In the last section, we explored Principal Com...",References:,[]


In [6]:
df = df.explode('concepts')
display(df.head(20))

,Unit,Module,Title,Text,Subheaders,concepts
0,Deep Learning and Model Deployment,Model Deployment,Quiz 10,NaN,NaN,Data science
0,Deep Learning and Model Deployment,Model Deployment,Quiz 10,NaN,NaN,Knowledge assessment
0,Deep Learning and Model Deployment,Model Deployment,Quiz 10,NaN,NaN,Key concepts
0,Deep Learning and Model Deployment,Model Deployment,Quiz 10,NaN,NaN,Data analysis
0,Deep Learning and Model Deployment,Model Deployment,Quiz 10,NaN,NaN,Statistical modeling
0,Deep Learning and Model Deployment,Model Deployment,Quiz 10,NaN,NaN,Machine learning
0,Deep Learning and Model Deployment,Model Deployment,Quiz 10,NaN,NaN,Data visualization
0,Deep Learning and Model Deployment,Model Deployment,Quiz 10,NaN,NaN,Predictive analytics
0,Deep Learning and Model Deployment,Model Deployment,Quiz 10,NaN,NaN,Big data
0,Deep Learning and Model Deployment,Model Deployment,Quiz 10,NaN,NaN,Data mining


In [15]:
df = df[50:60]

In [26]:
questions = []
answers = []
contexts = []

In [27]:
for i in range(len(df)):
    question = create_question(df.iloc[i])
    questions.append(question.split('\n')[0])
    answers.append(question.split('\n')[1])
    contexts.append(df.iloc[i]['Text'])
    print(i)

0
1
2


IndexError: list index out of range

In [23]:
print(question.split("\n")[0])

Q: What is the purpose of reserving a random portion of the dataset for testing in the model selection process?


In [26]:
text = df.iloc[40]['Text']

In [29]:
rqugescore = load("alirezamsh/rquge")

results = rqugescore.compute(generated_questions=questions[0:3], contexts=contexts, answers=answers)
print(results)

Token indices sequence length is longer than the specified maximum sequence length for this model (660 > 512). Running this sequence through the model will result in indexing errors


{'mean_score': 1.9705544312795003, 'instance_score': [2.8210134506225586, 1.9988523721694946, 1.0917974710464478]}


In [30]:
print(questions[0:3])

['Q: What is the purpose of splitting the dataset into train set and test set in the model selection procedure?', 'Q: What is the purpose of reserving a random portion of the dataset for testing in the model selection process?', 'Question: Why is it important to validate requirements during the solution development process?']


In [31]:
print(answers)

['A: The purpose of splitting the dataset into train set and test set is to replicate the setting of performing prediction on unseen data. The train set is used to train the candidate models, while the test set is used to evaluate their performance.', 'A: To replicate the setting of performing prediction on unseen data.', '']


In [36]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "Salesforce/mixqg-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

def run_qg(input_text, **generator_args):
    input_ids = tokenizer.encode(input_text, return_tensors="pt")
    generated_ids = model.generate(input_ids, **generator_args)
    return tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

run_qg(df.iloc[1]['Text'][0:1000], max_length=30, num_return_sequences=1, early_stopping=True)

['What is the best hyperparameters?']